In [ ]:
!pip install transformers
!pip install simpletransformers

In [ ]:
import pandas as pd
import numpy as np

from collections import defaultdict, Counter

from sklearn import model_selection
from sklearn.metrics import accuracy_score, classification_report,f1_score
from sklearn.model_selection import train_test_split

from simpletransformers.classification import ClassificationModel, ClassificationArgs

import nltk 
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:

import wandb
!wandb login 6fcfd33f292c19c9a44b565ee505839e5381b21e

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import shutil
dele = True

if dele:
  shutil.rmtree('/content/outputs')
  shutil.rmtree('/content/runs')
  shutil.rmtree('/content/cache_dir')
  shutil.rmtree('/content/wandb')


In [ ]:
# import modules and settings
import pandas as pd
import numpy as np
import glob
pd.options.display.max_colwidth =1000
# combine all the raw data to one single dataframe

path = r'/content/'
all_files = glob.glob(path + "/*.trans")
li = []

for filename in all_files:
    df = pd.read_csv(filename, sep='\t', names = ['txtID','txtStart','txtStop', 'speakerID','utterance' ])
    li.append(df)

rawdf = pd.concat(li, axis=0, ignore_index=True)
rawdf

In [ ]:
def catch_ai_type(df):
    
    '''
    1: creates a binary label actionItem if  occurs inutterance
    2: assigns an action item type and subtype for each utterance
    '''
    
    # binary labeling
    df["actionItem"]= df["utterance"].str.find("<") 
    df['actionItem'] = [False if x < 0 else True for x in df['actionItem']]
    
    # multiclas labeling
    df['aitype'] = 'noAI'
    df['aisubtype'] = 'noAI'
    
    for i, utterance in enumerate(df['utterance']):
        #Reminders
        if utterance.find('<create_single_reminder>') != -1:
            df.at[i, 'aitype'] = str('Reminder')
            df.at[i, 'aisubtype'] = str('create_single_reminder')
         
        #Calendar
        if utterance.find('<find_calendar_entry>') != -1:
            df.at[i, 'aitype'] = str('Calendar')
            df.at[i, 'aisubtype'] = str('find_calendar_entry')

        if utterance.find('<create_calendar_entry>') != -1:
            df.at[i, 'aitype'] = str('Calendar')
            df.at[i, 'aisubtype'] = str('create_calendar_entry')

        if utterance.find('<add_agenda_item>') != -1:
            df.at[i, 'aitype'] = str('Calendar')
            df.at[i, 'aisubtype'] = str('add_agenda_item')
            
        if utterance.find('<open_agenda>') != -1:
            df.at[i, 'aitype'] = str('Calendar')
            df.at[i, 'aisubtype'] = str('open_agenda')
            
        #Communication           
        if utterance.find('<send_email>') != -1:
            df.at[i, 'aitype'] = str('Communication')
            df.at[i, 'aisubtype'] = str('send_email')

        if utterance.find('<find_email>') != -1:
            df.at[i, 'aitype'] = str('Communication')
            df.at[i, 'aisubtype'] = str('find_email')

        if utterance.find('<make_call>') != -1:
            df.at[i, 'aitype'] = str('Communication')
            df.at[i, 'aisubtype'] = str('make_call')

        #Search
        if utterance.find('<search>') != -1:
            df.at[i, 'aitype'] = str('search')
            df.at[i, 'aisubtype'] = str('search')
            
        #OnDevice
        if utterance.find('<open_setting>') != -1:
            df.at[i, 'aitype'] = str('OnDevice')
            df.at[i, 'aisubtype'] = str('OnDevice')
    return df

            
            
            
dftagged = catch_ai_type(df = rawdf)       

def catch_ai_tag(df):
    
    '''
    1: searches for ai tags if a ai type is found
    2: saves ai tags with corresponding tags in df
    '''
    
    for i, utterance in enumerate(df['utterance']):
        #Reminders
        if utterance.find('<create_single_reminder>') != -1:
            for arg in ['contact_name','reminder_text', 'start_date', 'start_time']:
                if str(utterance).find('<'  + arg + '>') != -1:
                    n = len(arg)+2
                    st = str(utterance).find('<'  + arg + '>')
                    en = str(utterance).find('</' + arg + '>')
                    obs = utterance[st+n:en]
                    df.at[i, arg] = str(obs)
                    
                if str(utterance).find('<'  + arg + '>') == -1:
                    df.at[i,arg] = 'NA'
                    
        #Calendar
        if any(x in utterance for x in 
               ['<find_calendar_entry>','<create_calendar_entry>','<add_agenda_item>','<open_agenda>']):
            for arg in ['contact_name','entry_type', 'start_date','end_date', 'start_time','end_time',
                        'title','absolute_location','implicit_location', 'agenda_item']:
                if str(utterance).find('<'  + arg + '>') != -1:
                    n = len(arg)+2
                    st = str(utterance).find('<'  + arg + '>')
                    en = str(utterance).find('</' + arg + '>')
                    obs = str(utterance[st+n:en])
                    df.at[i, arg] = str(obs)
                    
                if str(utterance).find('<'  + arg + '>') == -1:
                    df.at[i,arg] = 'NA'
        
        
        #Communication
        if any(x in utterance for x in 
               ['<send_email>','<find_email>','<make_call>','<open_agenda>']):
            for arg in ['contact_name','email_subject', 'email_content','from_contact_name']:
                if str(utterance).find('<'  + arg + '>') != -1:
                    n = len(arg)+2
                    st = str(utterance).find('<'  + arg + '>')
                    en = str(utterance).find('</' + arg + '>')
                    obs = str(utterance[st+n:en])
                    df.at[i, arg] = str(obs)
                    
                if str(utterance).find('<'  + arg + '>') == -1:
                    df.at[i,arg] = 'NA'        

        #Search
        if utterance.find('<search>') != -1:
            for arg in ['query_term']:
                if str(utterance).find('<'  + arg + '>') != -1:
                    n = len(arg)+2
                    st = str(utterance).find('<'  + arg + '>')
                    en = str(utterance).find('</' + arg + '>')
                    obs = utterance[st+n:en]
                    df.at[i, arg] = str(obs)
                    
                if str(utterance).find('<'  + arg + '>') == -1:
                    df.at[i,arg] = 'NA'
            
        #ondevice
        if utterance.find('<open_setting>') != -1:
            for arg in ['setting_type']:
                if str(utterance).find('<'  + arg + '>') != -1:
                    n = len(arg)+2
                    st = str(utterance).find('<'  + arg + '>')
                    en = str(utterance).find('</' + arg + '>')
                    obs = utterance[st+n:en]
                    df.at[i, arg] = str(obs)
                    
                if str(utterance).find('<'  + arg + '>') == -1:
                    df.at[i,arg] = 'NA'
                    
    return df
       
    
    
    
dftagged = catch_ai_tag(df = dftagged)       

dfai = dftagged[dftagged['actionItem'] == True]

data = dftagged[['utterance', 'actionItem']]


# remove ai item labels
import re 
for i, utter in enumerate(data['utterance']):
    data.at[i, 'utter'] = re.sub("[\<\</].*?[\>\>]", "", utter)



/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [ ]:
data = data[data['utter'] != ""]
data = data[['utter', 'actionItem']]
data.head()

,utter,actionItem
0,"I, uh, had some communications with him yesterday or the day before. So, he doesn't plan on coming real soon, but he is taking sabbatical this year and so he'll probably come for six weeks at some point.",False
1,"Ah, so comfortable.",False
2,Smooth.,False
3,Mm-hmm.,False
4,Good.,False


In [ ]:
def tokenize(utter):
    tokens = [word_tokenize(t) for t in utter]
    return tokens

data['norm'] = tokenize(data['utter'])

def normalize(tokens,
             low = True,
             re_punct = False,
             re_dig = False,
             re_stop = False,
             re_small = False,
             lemma = False,
             to_str = True):
        
    
    tokens = [t.lower() for t in tokens]
    
    if re_punct:
        puncts = list(string.punctuation)+['..','...','<','>','--']
        tokens = [''.join([t for t in token if not t in puncts]) for token in tokens]
        tokens = [t for t in tokens if t]
        
    if re_dig:
        tokens = [''.join([t for t in token if not t.isdigit()]) for token in tokens]
        tokens = [t for t in tokens if t]
        
    if re_stop:
        stop_words = set(stopwords.words('english'))
        tokens = [t for t in tokens if not t in stop_words]
    
    if re_small:
        tokens = [t for t in tokens if not len(t)<=2]
        
    if lemma:
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(t) for t in tokens]
    
    return tokens
        

In [ ]:
import string
data['norm'] = data['norm'].apply(lambda x: normalize(x,
                                                      re_punct = True,
                                                      re_dig = True,
                                                      re_stop = False,
                                                      re_small = False,
                                                      lemma = True
                                                      ))

data['norm'] = [' '.join(l) for l in data['norm']]



In [ ]:
data[data['actionItem']!= False].head()

,utter,actionItem,norm
88,"Are we to r- Just to make sure I know what's going on, we're talking about Robert's thesis proposal today ? Is that true?",True,are we to r just to make sure i know what s going on we re talking about robert s thesis proposal today is that true
129,"Yeah, um, if you would have checked your email you may have received a note from Yees asking you to send me the, uh, up-to-d- current formalism thing that you presented .",True,yeah um if you would have checked your email you may have received a note from yees asking you to send me the uh uptod current formalism thing that you presented
150,"And y- uh, email any time , but most usefully before -",True,and y uh email any time but most usefully before
158,The twenty-ninth .,True,the twentyninth
161,That's when I'm meeting with Wolfgang Wahlster to sell him this idea .,True,that s when i m meeting with wolfgang wahlster to sell him this idea


In [ ]:
data = data[['norm', 'actionItem']]

In [ ]:
#data = data[data['utter'] != ""]
#data = data[['utter', 'actionItem']]
data.columns = ['text', 'labels']
X = data['text']
y = data['labels'].astype(int)


In [ ]:
data

,text,labels
0,"I, uh, had some communications with him yesterday or the day before. So, he doesn't plan on coming real soon, but he is taking sabbatical this year and so he'll probably come for six weeks at some point.",False
1,"Ah, so comfortable.",False
2,Smooth.,False
3,Mm-hmm.,False
4,Good.,False
...,...,...
21026,O_ two seven nine five one five zero eight two five four.,False
21027,Transcript number two seven seven one two seven nine zero. nine O_ nine eight nine zero one five zero one two two eight eight four five two five O_ O_ three eight two four five six zero zero eight eight one nine five six six,False
21028,O_ five O_ two one eight seven zero seven zero two nine one two three zero zero five three six three five two four seven seven six four eight six two eight zero nine.,False
21029,And we are -,False


In [ ]:

print(Counter(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2020, stratify = y)
train = pd.concat([X_train,y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)



Counter({0: 20647, 1: 384})


In [ ]:

print('Shape of train set {}'.format(train.shape))
print('Shape of test set {}'.format(test.shape))

train.isnull().sum().sum()
test.isnull().sum().sum()

print(pd.crosstab(y_train, columns='count'))
print(pd.crosstab(y_test, columns='count'))

Shape of train set (16824, 2)
Shape of test set (4207, 2)
col_0   count
labels       
0       16517
1         307
col_0   count
labels       
0        4130
1          77


In [ ]:
l = []

for i in (data['text']):
  l.append(len(i))

import numpy as np
print(np.median(l))
print(np.max(l))

24.0
1239


In [ ]:
model_args = ClassificationArgs()
from simpletransformers.classification import ClassificationModel, ClassificationArgs

# early stopping
#model_args.use_early_stopping = True
#model_args.early_stopping_delta = 0.001
#model_args.early_stopping_metric = "eval_tp"
#model_args.early_stopping_metric_minimize = False
#model_args.early_stopping_patience = 5
#model_args.evaluate_during_training_steps = 1000

# epochs
model_args.num_train_epochs=1
model_args.overwrite_output_dir=True
model_args.manual_seed=42

model_args.wandb_project = "vt1"

# numeric hyperparams
model_args.max_seq_length = 512 # Maximum sequence length the model will support. docs/usge
model_args.max_length = 1200 # The max length of the sequence to be generated. Between 0 and infinity. Default to 20.

model_args.sliding_window = Falses


# Training with sliding window




In [ ]:
#model_args = ClassificationArgs(num_train_epochs=1, overwrite_output_dir=True, manual_seed=2020)
model = ClassificationModel(model_type='roberta', model_name='roberta-base', use_cuda=True, num_labels=2, args=model_args)
model.train_model(train)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

wandb: Currently logged in as: loran_avci (use `wandb login --relogin` to force relogin)


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


(2103, 0.05343908816512344)

In [ ]:
result, model_outputs, wrong_preds = model.eval_model(test)



Training loss,0.0004
lr,0.0
global_step,2100
_step,41
_runtime,819
_timestamp,1605739927


Training loss,▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr,▄▇███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁
global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
predictions = []

for x in model_outputs:
    predictions.append(np.argmax(x))

print(classification_report(test['labels'], predictions),"\n")
print("Confusion Matrix: \n", confusion_matrix(test['labels'], predictions),"\n")
print("ROC AUC:", round(roc_auc_score(test['labels'], predictions),4))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4130
           1       0.88      0.79      0.84        77

    accuracy                           0.99      4207
   macro avg       0.94      0.90      0.92      4207
weighted avg       0.99      0.99      0.99      4207
 

Confusion Matrix: 
 [[4122    8]
 [  16   61]] 

ROC AUC: 0.8951


In [ ]:
result

{'eval_loss': 0.037253051087594,
 'fn': 16,
 'fp': 8,
 'mcc': 0.8340200775856744,
 'tn': 4122,
 'tp': 61}

In [ ]:
test['pred'] = predictions
test[test['pred'] != test['labels']]

,text,labels,pred
3661,"""Let's look at the belief-net"" O_K. So that means that I should switch to the other program.",1,0
3536,Can you maximize the window ?,1,0
19388,"I forget what it'll be, resource management, Wall Street Journal, something. Some - some read task actually, that they're - preparing.",1,0
3497,O_K. Have a great meeting. I'll - I'll come back up in about an hour and check and see if you're still meeting .,1,0
7029,"I studied Barry's sim code , more or less. to take the first step the spectral subtraction. and we have some - the feature for Italian database and we will try with this feature with the filter to find the result. But we haven't result until this moment.",0,1
7461,"And, somewhere is my agenda .",1,0
11906,For one of - one of .,0,1
3726,Unmark Time of Day,0,1
12749,So let me - let me make a note to do that.,1,0
15102,What was it that you had done last week when you showed - Do you remember?,1,0


In [ ]:
test[test['pred'] ==1]

,text,labels,pred
16106,The - the meeting is July sixteenth through eighteenth .,1,1
10947,"But, it's after the thresholding. So, maybe - maybe we might do it before , yeah .",1,1
8698,"Yeah something like that. Um, uh, you know, maybe - maybe we'll involve Dan Ellis at some - some level as well.",1,1
5804,"That was a question that was kind of raised by Mike Shire's thesis , and on - in that case in terms of reverberation .",1,1
19347,"Well actually, so this is a test that you should do then .",1,1
...,...,...,...
12372,It's KLEIST . It's the uh Bielefeld generation of uh spatial descriptions and whatever.,1,1
20378,"I could send Brian the, um - the transcript . I know these are - er, uh, I could send him that if it would be possible, or a good idea or not, to try to do a s- forced alignment on what we're - on the way we're encoding overlaps now.",1,1
8687,DARPA mee-,1,1
845,So I should still do that .,1,1
